<center>
<h1> TP 4 : Programmation Dynamique </h1>
<h1> Année 2021-2022 - 2e année département Sciences du Numérique </h1>
<h1> ANACLET Paul </h1> 
<h1> BONREPAUX Rémi </h1>    
</center>

## Récupération des données
Cette fonction permet de **lire les données** du fichier passé en paramètre.

In [1]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

## Résolution du problème du sac à dos

L’algorithme ci-dessous permet de résoudre le problème du sac à dos avec une programmation dynamique.

In [2]:
function solveSacADos(price, weight, capacity)
    nbObjet = length(price)
    C = zeros(nbObjet,capacity)
    solution_precedente =  Dict()
    solution_actuelle=  Dict()
    
# Initialisation la première ligne du tableau
    for j in 1:capacity
# Initialisation de la solution 
        solution_precedente[j] = []
        if j-weight[1] >= 0
            C[1,j] = price[1]
            push!(solution_precedente[j],1)
        end
    end
            
# Récurrence
    for i in 2:nbObjet      
        solution_actuelle = deepcopy(solution_precedente)  
        
        for j in 1:capacity
# Cas 1: l'objet peut rentrer dans le sac
            if j-weight[i] >= 0 
                if j-weight[i] == 0
# Cas 1.1: le poid de l'objet i est exactement la capacité maximale du sac
                    C[i,j] = max(C[i-1,j], price[i])
# Si on ne prend pas l'objet i, la solution ne change pas. Sinon on ne prend que l'objet i.
                    if max(C[i-1,j], price[i]) == price[i] 
                        solution_actuelle[j] = []
                        push!(solution_actuelle[j],i)
                    end
                else
# Cas 1.2: le poid de l'objet i est inférieur à la capacité maximale du sac
# Si on ne prend pas l'objet i, la solution ne change pas. Sinon on ajoute à la solution précedente l'objet i.
                    if max(C[i-1,j], C[i-1,j-weight[i]] + price[i]) == C[i-1,j-weight[i]] + price[i] 
                        solution_actuelle[j] = deepcopy(solution_precedente[j-weight[i]])
                        push!(solution_actuelle[j],i)
                    end
                    C[i,j] = max(C[i-1,j], C[i-1,j-weight[i]] + price[i])
                end
            else
                C[i,j] = C[i-1,j]
            end
        end
        solution_precedente = deepcopy(solution_actuelle)
    end
    sol = C[nbObjet,capacity]
    
    return sol,solution_actuelle[capacity]
end

solveSacADos (generic function with 1 method)

# Analyse des résultats

Le code ci-dessous permet d'obtenir les résultats de l'algorithme pour différentes données. Il suffit de décommenter le nom des fichiers que nous voulons tester. 

L'algorithme renvoi la liste sol où sont énumérés le numéro des objets à prendre. 
Afin d'excuter cette cellule, il est nécessaire de récuperer les fichiers de tests donnés sur moodle.

In [3]:
#Tests de performance sur les différents jeux de données
# Un fichier de chaque dossier
test = ["test.opb.txt","almost_strongly_correlated/knapPI_5_50_1000_1_-2096.opb.txt","circle/knapPI_16_20_1000_1_-2291.opb.txt" ,
"inverse_strongly_correlated/knapPI_4_50_1000_1_-994.opb.txt","profit_ceiling/knapPI_15_20_1000_5_-969.opb.txt","similar_weights/knapPI_9_50_1000_1_-995.opb.txt",
"strongly_correlated/knapPI_3_50_1000_1_-1894.opb.txt","subset_sum/knapPI_6_50_1000_1_-994.opb.txt","uncorrelated/knapPI_1_50_10000_1_-68946.opb.txt"] 


for t in test
    price, weight, capacity = readKnaptxtInstance("instancesETU/KNAPnewformat/"*t)

    println(" _______________________     Test ",t,"    ______________________")
    @time max,sol = solveSacADos(price, weight, capacity)
   
    println("")
    println(" MAXIMUM = ",  max)
    println(" solution = ",sol)
    println("\n")
end



 _______________________     Test test.opb.txt    ______________________
  0.466096 seconds (235.74 k allocations: 13.616 MiB, 99.84% compilation time)

 MAXIMUM = 65.0
 solution = Any[2, 4]


 _______________________     Test almost_strongly_correlated/knapPI_5_50_1000_1_-2096.opb.txt    ______________________
  0.275306 seconds (816.14 k allocations: 45.548 MiB, 5.44% gc time, 6.59% compilation time)

 MAXIMUM = 2096.0
 solution = Any[7, 11, 14, 24, 26, 36, 37, 38, 39, 45, 49]


 _______________________     Test circle/knapPI_16_20_1000_1_-2291.opb.txt    ______________________
  0.076145 seconds (283.25 k allocations: 15.792 MiB, 15.23% gc time)

 MAXIMUM = 2291.0
 solution = Any[3, 6, 15, 17]


 _______________________     Test inverse_strongly_correlated/knapPI_4_50_1000_1_-994.opb.txt    ______________________
  0.226400 seconds (736.37 k allocations: 38.708 MiB)

 MAXIMUM = 994.0
 solution = Any[38]


 _______________________     Test profit_ceiling/knapPI_15_20_1000_5_-969.opb.

## Résolution du problème du voyageur

In [2]:
function bellman_ford(Mat,ptsDebut,ptsFinal)
    # initialisation vecteur solution 
    taille = size(Mat, 1)
    Sol = zeros(size(Mat, 1)) .+ Inf
    Sol[ptsDebut] = 0
    # Chemin associer au vecteur solution
    Chemin = zeros(taille)

    #Sommet à exploré
    explo = ptsDebut

    #On s'arrete quand on a un chemin qui va au point finale (le chemin pour aller à F != Inf)
    while Sol[ptsFinal] == Inf 
        #On itére sur tout les sommets accèssible depuis le sommet à explorer
        for i in 1:taille
            if i != explo
                minimum = min(Sol[explo] + Mat[explo,i], Sol[i])
                if minimum != Sol[i]
                    Chemin[i] = explo
                end
                Sol[i] = minimum
            end
            
        end

        #On change le sommet à explorer
        explo = (explo + 1)
        if explo%taille == 0
            explo = explo % taille + 1
        end
    end


    #Remonter pour connaitre le chemin
    Chemin = Int.(Chemin)
    explo = Chemin[ptsFinal] #point finale
    CheminStr = string(ptsFinal)
    while explo != 0
        CheminStr = string(explo, " -> ",CheminStr )
        explo = Chemin[explo]
    end
    return Sol, CheminStr
end

bellman_ford (generic function with 1 method)

In [3]:
# Matrice du problème du cours 
CoutChemin = [0 3 Inf Inf 5 Inf;Inf 0 4 Inf Inf Inf; Inf Inf 0 2 Inf Inf; Inf Inf Inf 0 Inf 3;Inf -1 Inf 9 0 Inf;Inf Inf Inf Inf Inf 0]

# A = 1, B = 2 ...
ptsDepart = 1
ptsFin = 6

Sol, CheminStr = bellman_ford(CoutChemin,ptsDepart,ptsFin)


println("Coût du chemin :")
println(Sol[ptsFin])
println("Chemin a prendre:")
println(CheminStr)

Coût du chemin :
12.0
Chemin a prendre:
1 -> 2 -> 3 -> 4 -> 6


#### Amélioration possible:  
On pourrait améliorer la robustesse par exemple vérifier qu'il y a bien un chemin qui mène au point finale.  
On s'arrete quand la valeur pour arriver au point d'arriver n'est plus infinie. Cependant, rien ne garantie que se soit la meilleur.  